## Import libraries

In [1]:
import faiss
import numpy as np
import pandas as pd
from read_s3 import read_s3_file
import time

## Process data

In [2]:
BATCH_SIZE = 50000
BATCH_NUMBER = 10
BATCH_COUNT = 0
BATCH_COUNT_LIMIT = 10

df = pd.DataFrame(columns=["parent_asin", "embedding"])

# append all batches into one file
for i in range(BATCH_COUNT_LIMIT):
    key = f"data_vectors/file_0_7M_{BATCH_COUNT * BATCH_SIZE}_{(BATCH_COUNT + 1) * BATCH_SIZE - 1}"

    data = read_s3_file(key)

    if BATCH_COUNT == 0:
        df = data
    else:
        df = pd.concat([df, data], ignore_index=True)
    BATCH_COUNT += 1

## unpack embeddings into matrix

is_valid = df['embedding'].apply(lambda x: isinstance(x, np.ndarray) and x.shape == (384,))
df_clean = df[is_valid]

matrix = np.stack(df_clean['embedding'].values).astype('float32')

print(matrix.shape)
# (499971, 384)


(499971, 384)


## Initialize Index using FAISS

In [3]:
index = faiss.IndexFlatL2(384)
index.add(matrix)

## Query

In [ ]:
from sentence_transformers import SentenceTransformer

# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# prompt
query = "best eczema face wash for sensitive asian skin"

# convert query to vector
query_embedding = model.encode(query, convert_to_numpy=True).astype('float32')

# number of top results to retrieve
k = 100

## Search
D, I = index.search(query_embedding.reshape(1, -1), k)

# print("Distances:", D)
# print("Indices:", I)
# Retrieve the top-k results
top_k_results = df_clean.iloc[I[0]]

# top k parent_asins
top_k_parent_asins = top_k_results['parent_asin'].unique()

print("Top-k results:")
print(top_k_results[['parent_asin', 'embedding']])

## 0.3s

Top-k results:
       parent_asin                                          embedding
311829  B09WYZJVFH  [0.0004865061, 0.059590034, 0.05439229, 0.0282...
179787  B000052YN9  [0.028374797, 0.13186209, 0.049382284, -0.0146...
62631   B00IFWC058  [-0.0064997077, 0.0636541, 0.05518529, -0.0043...
260439  B00SBSBI7M  [0.016015513, 0.077947654, 0.035339646, 0.0394...
89826   B0C5FJT32P  [-0.014239035, 0.17768838, 0.091014564, -0.012...
...            ...                                                ...
101747  B083M54MZL  [-0.08751378, -0.00069189805, 0.08085996, -0.0...
332462  B079SX37P4  [-0.050606657, 0.046125174, 0.07001063, 0.0504...
252408  B0C6BVZ7ZW  [-0.037568472, 0.040740255, 0.052189335, 0.065...
292970  B0C5S27LN6  [-0.057144377, 0.062027723, 0.09078525, 0.0511...
92      B000GEZTLK  [-0.044107404, 0.05610219, 0.064003825, -0.002...

[100 rows x 2 columns]


## Validate

In [16]:
# function to return bayesian average rating
def bayesian_avg_rating(df):

    metadata_top_k = df.copy()

    metadata_top_k.columns = metadata_top_k.columns.str.lower()
    
    # https://www.algolia.com/doc/guides/managing-results/must-do/custom-ranking/how-to/bayesian-average/
    # C (confidence number): 25% percentile
    C = metadata_top_k['average_rating'].quantile(0.25)
    # m: smoothing factor -- median of rating_number
    m = metadata_top_k['rating_number'].median()

    metadata_top_k['bayes_average_rating'] = (
        (metadata_top_k['average_rating'] * metadata_top_k['rating_number'] + C * m) /
        (metadata_top_k['rating_number'] + m)
    )

    # sort by bayesian average rating and return top 5
    metadata_top_k_sorted = metadata_top_k.sort_values(by='bayes_average_rating', ascending=False)

    return metadata_top_k_sorted

### Connect from S3 Parquet

In [ ]:
#### Read parquet from S3 method
start_time = time.time()

# retrieve meta data for product names
metadata_key = 'meta_data.parquet'

metadata = read_s3_file(metadata_key)

# search relevant products
metadata_top_k = metadata[metadata['parent_asin'].isin(top_k_parent_asins)]

# calculate and sort by bayes avg rating
metadata_top_k_sorted = bayesian_avg_rating(metadata_top_k)

print(f"final time: {time.time() - start_time:.2f} seconds")
metadata_top_k_sorted.head(5)


final time: 12.88 seconds


,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author,bayes_average_rating
179752,All Beauty,Aveeno Daily Moisturizing Lotion & Wash Regime...,4.7,44872,[2 item Aveeno Daily Moisturizing Lotion and B...,"[Gently cleanse and nourish dry, sensitive ski...",19.57,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': ['Aveeno Daily Moisturizer, Body Lot...",Aveeno,"[Beauty & Personal Care, Skin Care, Body, Mois...","{""Package Dimensions"": ""11.34 x 5.31 x 4.17 in...",B0C6BVZ7ZW,None,None,None,4.690466
465920,All Beauty,"NOW Solutions, Grapeseed Oil, Skin Care for Se...",4.7,21373,[SENSITIVE SKIN CARE: 100% Pure Grapeseed Oil ...,"[100% Pure Grapeseed Oil is a light, odorless ...",12.45,{'hi_res': ['https://m.media-amazon.com/images...,{'title': ['Want That Kardashian Skin Glow. Ha...,NOW,"[Beauty & Personal Care, Skin Care, Body, Mois...","{""Brand"": ""NOW"", ""Scent"": ""Grape Seed O"", ""Ite...",B09KX3VVBH,None,None,None,4.680328
183642,All Beauty,Sky Organics Organic Lip Balms With Beeswax fo...,4.6,14795,[WHY WE LOVE IT: Sky Organics Organic Lip Balm...,[],6.88,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Sky Organics,"[Beauty & Personal Care, Skin Care, Lip Care, ...","{""Brand"": ""Sky Organics"", ""Item Form"": ""Stick""...",B0BR7SG5MH,None,None,None,4.576658
746593,All Beauty,"Elizabeth Taylor Women's Perfume, Passion, Eau...",4.6,5371,[Passion by Elizabeth Taylor is as fiery and a...,"[Passion By Elizabeth Taylor For Women, Eau De...",21.33,{'hi_res': ['https://m.media-amazon.com/images...,{'title': ['Women's Perfume by Elizabeth Taylo...,Elizabeth Taylor,"[Beauty & Personal Care, Fragrance, Women's, E...","{""Brand"": ""Elizabeth Taylor"", ""Item Form"": ""Sp...",B0CFGDPNCH,None,None,None,4.540571
725700,All Beauty,Lubriderm Daily Moisture Hydrating Unscented B...,4.7,1734,[3-fluid ounce tube of Lubriderm Daily Moistur...,"[Ideal for normal-to-dry skin types, Lubriderm...",None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': ['Style Code Live: Tika Sumpter', 'L...",Lubriderm,"[Beauty & Personal Care, Skin Care, Body, Mois...","{""Brand"": ""Lubriderm"", ""Scent"": ""Fragrance Fre...",B005IHSTRQ,None,None,None,4.523185


### Connect from Snowflake

In [17]:
import snowflake.connector as sfcon
from dotenv import load_dotenv
import os

load_dotenv()

snowflake_account  = os.getenv('SNOWFLAKE_ACCOUNT')
snowflake_user = os.getenv('SNOWFLAKE_USER')
snowflake_password = os.getenv('SNOWFLAKE_PASSWORD')
snowflake_db= os.getenv('SNOWFLAKE_DB')
snowflake_wh= os.getenv('SNOWFLAKE_WH')
snowflake_schema = os.getenv('SNOWFLAKE_SCHEMA')
snowflake_meta_table = os.getenv('SNOWFLAKE_META_TABLE')

# establish con
conn = sfcon.connect(
    user=snowflake_user,
    password=snowflake_password,
    account=snowflake_account,
    database=snowflake_db,
    warehouse=snowflake_wh,
    schema=snowflake_schema,
    meta_table=snowflake_meta_table
)

# create cursor
cursor = conn.cursor()

placeholders = ', '.join(['%s' for _ in top_k_parent_asins])

# query Snowflake
results = cursor.execute(
    f'''
    SELECT
        average_rating,
        ARRAY_TO_STRING(categories, ',') as categories_clean,
        ARRAY_TO_STRING(description, ',') as description_clean,
        REPLACE(details, '\\n', '') as details,
        ARRAY_TO_STRING(features, ',') as features_clean,
        images :: string as images,
        main_category,
        parent_asin,
        price,
        rating_number,
        store,
        title,
        videos :: string as videos    
    FROM META_DATA_TABLE
    WHERE parent_asin in ({placeholders})
    ''',
    top_k_parent_asins.tolist()
).fetchall()

# retrieve column names
results_column_name = cursor.execute(
    f'''
    SELECT COLUMN_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_SCHEMA = '{snowflake_schema}'  
    AND TABLE_NAME = '{snowflake_meta_table}'
    ORDER BY ORDINAL_POSITION
    '''
).fetchall()

results_column_names = [col[0] for col in results_column_name]

# convert to df
results_df = pd.DataFrame(results, columns=results_column_names)

# sort by bayesian average rating
snowflake_top_k_sorted = bayesian_avg_rating(results_df)

# close cursor and connection
cursor.close()
conn.close()

# results_df
snowflake_top_k_sorted

,average_rating,categories,description,details,features,images,main_category,parent_asin,price,rating_number,store,title,videos,bayes_average_rating
17,4.8,"Beauty & Personal Care,Skin Care,Face,Creams &...","Developed with dermatologists, CeraVe Moisturi...","{""Brand"": ""CeraVe"", ""Scent"": ""Fragrance Free"",...",[ HYALURONIC ACID MOISTURIZER ] With hyaluroni...,"{""hi_res"":[""https://m.media-amazon.com/images/...",All Beauty,B08KVK68F7,17.78,117967,CeraVe,CeraVe Moisturizing Cream | Body and Face Mois...,"{""title"":[""Watch before you buy | Honest Revie...",4.794998
56,4.8,"Beauty & Personal Care,Skin Care,Face,Cleanser...","Cetaphil Gentle Cleansing Bar, Hypoallergenic,...","{""Item Form"": ""Bar"", ""Skin Type"": ""Sensitive, ...",Free of any harsh detergents that may irritate...,"{""hi_res"":[""https://m.media-amazon.com/images/...",All Beauty,B0B4H48J4J,5.49,16565,Cetaphil,"CETAPHIL Gentle Cleansing Bar, 4.5 oz , Nouris...","{""title"":[""Leaves Your Face Feeling Clean"",""Be...",4.766908
50,4.8,"Beauty & Personal Care,Skin Care,Body,Cleanser...",Dove Sensitive Skin Beauty Bar combines classi...,"{""Item Form"": ""Bar"", ""Skin Type"": ""Sensitive"",...",GENTLE ON SENSITIVE SKIN: Nourish your skin wi...,"{""hi_res"":[""https://m.media-amazon.com/images/...",All Beauty,B07X1VDZ73,15.74,14446,Dove,Dove Beauty Bar More Moisturizing Than Bar Soa...,"{""title"":[""Honest Review Dove Beauty Bar"",""In ...",4.762509
18,4.8,"Beauty & Personal Care,Skin Care,Body,Cleanser...","If you have sensitive skin, you know that it n...","{""Item Form"": ""Lotion"", ""Skin Type"": ""Sensitiv...",Dove Body Wash for Sensitive Skin gently clean...,"{""hi_res"":[""https://m.media-amazon.com/images/...",All Beauty,B08373D96S,None,11698,Dove,Dove Hypoallergenic Body Wash To Moisturize Se...,"{""title"":[""Its Real Not Fake !!!"",""Feels amazi...",4.754700
27,4.8,"Beauty & Personal Care,Skin Care,Body,Cleanser...",Nourish your skin senses with Dove Hypoallerge...,"{""Item Form"": ""Gel"", ""Skin Type"": ""Sensitive"",...",24HR RENEWING MICROMOISTURE: Dove Hypoallergen...,"{""hi_res"":[""https://m.media-amazon.com/images/...",All Beauty,B0BY5NX1BK,None,5005,Dove,Dove Body Wash With Pump Sensitive Skin 3 Coun...,"{""title"":[""Dove Sensitive Skin- Watch before y...",4.708047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,4.2,"Beauty & Personal Care,Skin Care,Face,Cleanser...",Eucerin Redness Relief Soothing Cleanser gentl...,"{""Item Form"": ""Gel"", ""Skin Type"": ""Sensitive"",...","ENRICHED: With Licochalcone, a skin soothing e...","{""hi_res"":[""https://m.media-amazon.com/images/...",All Beauty,B006R6VJBK,None,292,Eucerin,Eucerin Sensitive Skin Redness Relief Cleansin...,"{""title"":[""Eucerin Redness Relief Cleansing Ge...",4.367301
6,4.2,"Beauty & Personal Care,Skin Care,Face,Polishes...",,"{""Brand"": ""Clean & Clear"", ""Item Form"": ""Gel"",...",5-ounce of Clean & Clear Oil-Free Deep Action ...,"{""hi_res"":[""https://m.media-amazon.com/images/...",All Beauty,B0056AY5GY,None,642,Clean & Clear,Clean & Clear Oil-Free Deep Action Exfoliating...,"{""title"":[],""url"":[],""user_id"":[]}",4.339888
66,4.3,"Beauty & Personal Care,Skin Care,Body,Moisturi...",E45 Cream is a trusted formula suitable for th...,"{""Brand"": ""E45"", ""Scent"": ""Eucalyptus"", ""Item ...","Effective, non-greasy emollient is quickly and...","{""hi_res"":[""https://m.media-amazon.com/images/...",Health & Personal Care,B004AV1J2S,32.02,3456,E45,"E45 Eucalyptus Scented Cream, 500g - Whole Bod...","{""title"":[],""url"":[],""user_id"":[]}",4.330182
79,4.3,"Beauty & Personal Care,Skin Care,Body,Cleanser...",TWO BARS,"{""Item Form"": ""Bar"", ""Skin Type"": ""Dry"", ""Bran...","For dry, irritated or damaged skin - won't clo...","{""hi_res"":[null],""large"":[""https://m.media-ama...",All Beauty,B012YJZPN6,12.99,4194,SOAP WORKS,Soap Works Emu Oil Soap Bar 2 BARS (110g) Brand,"{""title"":[""Good for skin sensitive or itchy""],...",4.326266
